# **Import Object**

In [1]:
from datetime import datetime as dt
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import get_x_y

# **Load Energy**

In [2]:
energy_8_10_60m = get_x_y.get_target_energy(house_id=3,
                                            target_months=[8, 9, 10],
                                            intervals=3600)
energy_12_1_60m = get_x_y.get_target_energy(3, [12, 1], 3600)

energy_8_10_30m = get_x_y.get_target_energy(3, [8, 9, 10], 1800)
energy_12_1_30m = get_x_y.get_target_energy(3, [12, 1], 1800)

2012-10-15:FileNotFoundError
skip this date
2012-10-16:FileNotFoundError
skip this date
2012-10-17:FileNotFoundError
skip this date
2012-10-18:FileNotFoundError
skip this date
2012-10-19:FileNotFoundError
skip this date
2012-10-20:FileNotFoundError
skip this date
2012-10-21:FileNotFoundError
skip this date
2012-10-22:FileNotFoundError
skip this date
2012-10-23:FileNotFoundError
skip this date
2012-10-24:FileNotFoundError
skip this date
2012-10-25:FileNotFoundError
skip this date
2012-10-26:FileNotFoundError
skip this date
2012-10-27:FileNotFoundError
skip this date
2012-10-28:FileNotFoundError
skip this date
2012-10-29:FileNotFoundError
skip this date
2012-10-30:FileNotFoundError
skip this date
2012-10-31:FileNotFoundError
skip this date
2012-12-01:FileNotFoundError
skip this date
2012-12-02:FileNotFoundError
skip this date
2012-12-03:FileNotFoundError
skip this date
2012-12-04:FileNotFoundError
skip this date
2012-12-05:FileNotFoundError
skip this date
2012-10-15:FileNotFoundError
ski

# **Load Occupancy**

In [3]:
occupancy_sum = pd.read_csv('./ecodataset/Occupancy/03_summer.csv')
occupancy_sum = occupancy_sum.set_index('Unnamed: 0')

occupancy_win = pd.read_csv('./ecodataset/Occupancy/03_winter.csv')
occupancy_win = occupancy_win.set_index('Unnamed: 0')

In [4]:
occupancy_sum = get_x_y.to_intervals_occupancy(occupancies=occupancy_sum,
                                               intervals=1800)
occupancy_win = get_x_y.to_intervals_occupancy(occupancy_win, 1800)

In [5]:
occupancy_sum = occupancy_sum.T[10:].T
occupancy_win = occupancy_win.T[3:].T
# sliced periods don't have data for 2 weeks before

# **Create Ratio**

In [6]:
ratio_sum = get_x_y.build_ratio(date_columns=occupancy_sum.columns,
                                energies=energy_8_10_60m)
ratio_win = get_x_y.build_ratio(occupancy_win.columns, energy_12_1_60m)

# **Create Features**

In [7]:
energy_sum = get_x_y.get_corresponding_energy(
    occupancy_columns=occupancy_sum.columns,
    energies=energy_8_10_30m)

In [8]:
energy_win = get_x_y.get_corresponding_energy(occupancy_win.columns,
                                              energy_12_1_30m)

In [9]:
features_sum = pd.DataFrame()
features_sum["Energy"] = energy_sum
features_sum["Season"] = 1
features_sum["Time"] = [t for t in range(0, 48, 1)] * len(occupancy_sum.columns)
features_sum = features_sum[(features_sum.Time >= 12) & (features_sum.Time < 44)]

features_win = pd.DataFrame()
features_win["Energy"] = energy_win
features_win["Season"] = 0
features_win["Time"] = [t for t in range(0, 48, 1)] * len(occupancy_win.columns)
features_win = features_win[(features_win.Time >= 12) & (features_win.Time < 44)]

In [10]:
occupancy_sum = occupancy_sum[12:44].values.T.reshape(-1)
occupancy_sum = pd.DataFrame(occupancy_sum, columns=["Occupancy"])

occupancy_win = occupancy_win[12:44].values.T.reshape(-1)
occupancy_win = pd.DataFrame(occupancy_win, columns=["Occupancy"])

In [11]:
X = pd.concat([features_sum, features_win], axis=0)
Y = pd.concat([occupancy_sum, occupancy_win], axis=0)

In [12]:
X.to_csv("3_X_train.csv", index=False)
Y.to_csv("3_Y_train.csv", index=False)